In [1]:
!pip install numpy tensorflow
import numpy as np
import tensorflow as tf
from tensorflow import keras

You should consider upgrading via the '/usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [3]:
reconstructed_model = keras.models.load_model("model_v1")
# reconstructed_model.evaluate()

2022-06-02 19:19:15.832965: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
image = tf.keras.preprocessing.image
preprocess_input = tf.keras.applications.inception_v3.preprocess_input
vgg16 = tf.keras.applications.VGG16(include_top=False)


In [6]:
path = "/Users/hariangr/Documents/MyFiles/Developer/Bangkit/ml/notebook/samples/Asli NSFW (2).png"
img = image.load_img(path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
x = preprocess_input(img)
x.shape

(1, 224, 224, 3)

In [22]:
img_array = np.array([x])

In [213]:
X = vgg16.predict(img_array, batch_size=1, verbose=1)
X = X.reshape(1, 512, -1)

1/1 [==============================] - 0s 100ms/step


In [224]:
infer = reconstructed_model.predict(x)
# infer.reshape((1,))[0]

1/1 [==============================] - 0s 109ms/step


In [215]:
if infer < 0.5:
    print("Not safe")
else:
    print("Safe")

Not safe


In [225]:
infer.numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [ ]:
img_array.tolist()

In [14]:
img_array.tolist()

AttributeError: 'list' object has no attribute 'tolist'

In [27]:
import json
import base64
import tensorflow as tf

IMAGE_URI='gs://cloud-samples-data/ai-platform/built-in/image/tutorial_examples/daisy.jpg'

with tf.gfile.Open(IMAGE_URI, 'rb') as image_file:
  encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

image_bytes = {'b64': str(encoded_string)}
instances = {'image_bytes': image_bytes, 'key': '1'}
with open("prediction_instances.json","w") as f:
  f.write(json.dumps(instances))

AttributeError: module 'tensorflow' has no attribute 'gfile'

In [32]:
!pip install google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 132.1 kB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 79.9 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 KB 196.7 kB/s eta 0:00:001m193.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 KB 306.4 kB/s eta 0:00:001m307.4 kB/s eta 0:00:01
You should consider upgrading via the '/usr/local/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [2]:
import googleapiclient.discovery
def predict_json(project, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the ML Engine service object.
    # To authenticate set the environment variable
    # GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

predict_json("semiotic-joy-349807", "inappropriate_image", )

DefaultCredentialsError: The file prediction_instances.json does not have a valid type. Type is None, expected one of ('authorized_user', 'service_account', 'external_account', 'impersonated_service_account').